In [1]:
import os, pandas as pd
from dotenv import load_dotenv
from utility import Utils as U
import sqlalchemy

#### Pull tables to staging schema

In [2]:
load_dotenv()
conn = U.create_db_connection(os.getenv('username'), os.getenv('password'),os.getenv('db_host'),os.getenv('db_name') )

In [5]:
Countries_df = U.fetch_db_table('Countries', conn, 'Staging')
Themes_df = U.fetch_db_table('Themes', conn, 'Staging')
Regions_df = U.fetch_db_table('Regions', conn, 'Staging')
Recommendations_df = U.fetch_db_table('Recommendations', conn, 'Staging')
tags_df = U.fetch_db_table('Tags', conn, 'Staging')
tags_matrix = U.fetch_db_table('TagsMatrix', conn, 'Staging')
UPR_cycle = U.fetch_db_table('UPR_Cycle', conn, 'Staging')

In [6]:
Countries_df

,Country id,Country name,Regions
0,1,Uganda,Africa
1,2,Venezuela (Bolivarian Republic of),Latin America and the Caribbean
2,3,Hungary,Eastern Europe
3,4,Ireland,Western Europe & Others
4,5,Thailand,Asia-Pacific
...,...,...,...
144,145,Togo,Africa
145,146,Montenegro,Eastern Europe
146,147,Chad,Africa
147,148,Oman,Asia-Pacific


In [7]:
Regions_df

,Region id,Regions
0,1,Africa
1,2,Latin America and the Caribbean
2,3,Eastern Europe
3,4,Western Europe & Others
4,5,Asia-Pacific
5,6,Missing data


In [8]:
Countries_df = Countries_df.merge(Regions_df, left_on="Regions", right_on="Regions")
Countries_df = Countries_df.drop(columns=["Regions"])

In [9]:
Countries_df

,Country id,Country name,Region id
0,1,Uganda,1
1,6,United Republic of Tanzania,1
2,7,Eswatini,1
3,15,Namibia,1
4,16,Sierra Leone,1
...,...,...,...
144,132,Saudi Arabia,5
145,133,Viet Nam,5
146,135,State of Palestine*,5
147,148,Oman,5


In [10]:
Country_to_recommendations = UPR_cycle[['Countries Concerned','UPR Recommending States']]
Country_to_recommendations.index = Country_to_recommendations.index.set_names(['Recommendation_id'])
Country_to_recommendations = Country_to_recommendations.reset_index(level=0).drop_duplicates(subset='Recommendation_id', keep='first')
Country_to_recommendations = Country_to_recommendations.merge(Countries_df[["Country name","Country id"]], left_on='Countries Concerned', right_on='Country name')
Country_to_recommendations = Country_to_recommendations.drop(columns=['Countries Concerned','Country name']).rename(columns={"Country id": "Countries Concerned"})
Country_to_recommendations = Country_to_recommendations.merge(Countries_df[["Country name","Country id"]], left_on='UPR Recommending States', right_on='Country name')
Country_to_recommendations = Country_to_recommendations.drop(columns=['UPR Recommending States','Country name']).rename(columns={"Country id": "UPR Recommending States"})

In [11]:
Recommendation_to_theme = UPR_cycle[['Themes']]
Recommendation_to_theme.index = Recommendation_to_theme.index.set_names(['Recommendation id'])
Recommendation_to_theme = Recommendation_to_theme.reset_index(level=0)

In [12]:
Recommendation_to_theme = Recommendation_to_theme.merge(Themes_df, left_on='Themes', right_on='Themes')
Recommendation_to_theme = Recommendation_to_theme.drop(columns=['Themes']).rename(columns={'Theme id':'Themes'})

In [13]:
tags_to_themes = pd.melt(tags_matrix, value_vars=tags_matrix.columns.tolist(), var_name='Tag', value_name='Theme')
tags_to_themes = tags_to_themes.merge(tags_df[["Tags","Tag id"]], left_on='Tag', right_on='Tags').drop(columns=['Tags', 'Tag'])
tags_to_themes = tags_to_themes.merge(Themes_df[["Themes","Theme id"]], left_on='Theme', right_on='Themes').drop(columns=['Themes', 'Theme'])

### Normalized tables

### Join tables

In [14]:
Country_to_recommendations

,Recommendation_id,Countries Concerned,UPR Recommending States
0,0,1,70
1,1,1,70
2,2,1,70
3,46,11,70
4,335,28,70
...,...,...,...
1534,1244,70,68
1535,1245,70,68
1536,1367,73,149
1537,1368,73,149


In [15]:
Recommendation_to_theme

,Recommendation id,Themes
0,0,1
1,13,1
2,17,1
3,26,1
4,57,1
...,...,...
1534,1173,65
1535,1222,66
1536,1227,66
1537,1380,67


In [16]:
tags_to_themes

,Tag id,Theme id
0,2,7
1,2,45
2,3,36
3,4,8
4,5,17
5,6,57
6,7,28
7,8,41
8,12,41
9,9,12


#### Upload tables to Production schema

In [17]:
U.upload_data(Countries_df, conn, 'Countries', 'Production')
U.upload_data(Themes_df, conn, 'Themes', 'Production')
U.upload_data(Regions_df, conn, 'Regions', 'Production')
U.upload_data(Recommendations_df, conn, 'Recommendations', 'Production')
U.upload_data(tags_df, conn, 'Tags', 'Production')
U.upload_data(tags_matrix, conn, 'TagsMatrix', 'Production')
U.upload_data(UPR_cycle, conn, 'UPR_Cycle','Production')

539

In [18]:
U.upload_data(Country_to_recommendations, conn, 'Country_to_recommendations', 'Production')
U.upload_data(Recommendation_to_theme, conn, 'Recommendation_to_theme', 'Production')
U.upload_data(tags_to_themes, conn, 'tags_to_themes', 'Production')

48